<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">COLLABORATIVE FILTERING</font></h1>

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.

<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Acquiring the Data</a></li>
        <li><a href="#ref2">Preprocessing</a></li>
        <li><a href="#ref3">Collaborative Filtering</a></li>
    </ol>
</div>
<br>
<hr>



<a id="ref1"></a>
# Acquiring the Data

Now you're ready to start working with the data!

<hr>

<a id="ref2"></a>
# Preprocessing

First, let's get all of the imports out of the way:

In [1]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read each file into their Dataframes:

In [2]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('ml-latest/movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ml-latest/ratings.csv')

Let's also take a peek at how each of them are organized:

In [3]:
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies_df[movies_df.title.str.lower().str.contains('panda')  ]

,movieId,title,genres
144,146,"Amazing Panda Adventure, The (1995)",Adventure|Children
4291,4385,Pandaemonium (2000),Drama
12706,59784,Kung Fu Panda (2008),Action|Animation|Children|Comedy|IMAX
13572,67295,Kung Fu Panda: Secrets of the Furious Five (2008),Action|Animation|Children|Comedy
15206,77412,Panda! Go Panda! (Panda kopanda) (1972),Animation|Children
17313,87222,Kung Fu Panda 2 (2011),Action|Adventure|Animation|Children|Comedy|IMAX
18441,91823,Kung Fu Panda Holiday Special (2010),Animation|Children|Comedy
27501,127300,Panda! Go Panda! Rainy Day Circus (1973),Animation|Children
30040,136556,Kung Fu Panda: Secrets of the Masters (2011),Animation|Children
33654,149406,Kung Fu Panda 3 (2016),Action|Adventure|Animation


So each movie has a unique ID, a title with its release year along with it (Which may contain unicode characters) and several different genres in the same field. Let's remove the year from the title column and place it into its own one by using the handy [extract](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html#pandas.Series.str.extract) function that Pandas has.

Let's remove the year from the __title__ column by using pandas' replace function and store in a new __year__ column.

In [5]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

Let's look at the result!

In [6]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


With that, let's also drop the genres column since we won't need it for this particular recommendation system.

In [7]:
#Dropping the genres column
movies_df = movies_df.drop('genres', 1)

Here's the final movies dataframe:

In [8]:
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


<br>

Next, let's look at the ratings dataframe.

In [9]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.

In [10]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)

Here's how the final ratings Dataframe looks like:

In [11]:
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<hr>

<a id="ref3"></a>
# Collaborative Filtering

Now, time to start our work on recommendation systems. 

The first technique we're going to take a look at is called __Collaborative Filtering__, which is also known as __User-User Filtering__. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the __Pearson Correlation Function__.

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")


The process for creating a User Based recommendation system is as follows:
- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbours 
- Get the watched movie record of the user for each neighbour.
- Calculate a similarity score using some formula
- Recommend the items with the highest score


Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .

In [12]:
userInput = [
            {'title':'Lord of the Rings, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':2},
            {'title':'Lord of the Rings: The Two Towers, The', 'rating':5},
    {'title':'John Wick', 'rating':5},
    {'title':'Hobbit: An Unexpected Journey, The', 'rating':5},
    {'title':'Inception', 'rating':5},
    {'title':'Kung Fu Panda', 'rating':4.7},
    {'title':'Kung Fu Panda: Secrets of the Furious Five', 'rating':4.7},
    {'title':'Kung Fu Panda 3', 'rating':4.7}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Lord of the Rings, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,2.0
4,"Lord of the Rings: The Two Towers, The",5.0
5,John Wick,5.0
6,"Hobbit: An Unexpected Journey, The",5.0
7,Inception,5.0
8,Kung Fu Panda,4.7
9,Kung Fu Panda: Secrets of the Furious Five,4.7


#### Add movieId to input user
With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [13]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,2.0
3,2116,"Lord of the Rings, The",5.0
4,5952,"Lord of the Rings: The Two Towers, The",5.0
5,59784,Kung Fu Panda,4.7
6,67295,Kung Fu Panda: Secrets of the Furious Five,4.7
7,79132,Inception,5.0
8,98809,"Hobbit: An Unexpected Journey, The",5.0
9,115149,John Wick,5.0


#### The users who has seen the same movies
Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.


In [25]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
199,5,79132,3.0
251,7,79132,5.0
479,13,2,2.0
681,14,296,2.0


We now group up the rows by user ID.

In [26]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

lets look at one of the users, e.g. the one with userID=1130

In [27]:
userSubsetGroup.get_group(1130)

,userId,movieId,rating
104167,1130,1,0.5
104168,1130,2,4.0
104214,1130,296,4.0
104785,1130,5952,2.0
105004,1130,59784,2.0
105069,1130,79132,2.5


Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [28]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

Now lets look at the first user

In [29]:
userSubsetGroup[0:3]

[(2569,         userId  movieId  rating
  233940    2569        1     4.0
  233941    2569        2     5.0
  233993    2569      296     5.0
  234145    2569     2116     5.0
  234231    2569     5952     5.0
  234460    2569    59784     5.0
  234500    2569    67295     4.0
  234584    2569    79132     5.0
  234786    2569    98809     5.0
  234936    2569   115149     4.5), (36946,          userId  movieId  rating
  3402635   36946        1     5.0
  3402636   36946        2     3.0
  3402781   36946      296     4.0
  3403471   36946     2116     2.5
  3404672   36946     5952     4.0
  3405838   36946    59784     5.0
  3405982   36946    67295     4.0
  3406174   36946    79132     4.0
  3406489   36946    98809     1.5
  3406733   36946   115149     3.0), (96884,          userId  movieId  rating
  8968647   96884        1     3.5
  8968648   96884        2     3.5
  8968683   96884      296     3.5
  8968853   96884     2116     2.5
  8969089   96884     5952     5.0
  8969425

#### Similarity of users to input user
Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.  
we're going to find out how similar each user is to the input through the __Pearson Correlation Coefficient__. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below. 

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation. 

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.

In [21]:
#userSubsetGroup = userSubsetGroup[0:500]

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient


In [30]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [31]:
pearsonCorrelationDict.items()

dict_items([(2569, -0.04739508007730602), (36946, -0.16591832104781726), (96884, 0.431032749221613), (112293, 0.28027451765096756), (138980, -0.2610317471200417), (140615, -0.1445314571618524), (147003, -0.200962266388885), (153441, 0.252840467657658), (1950, -0.2896620610108239), (4208, 0.5394809991873387), (8303, 0.12027018136999794), (9101, 0.4617947057304276), (9880, 0.4232889865799358), (11357, 0.3310423554409517), (12354, 0.03060666838914018), (15895, 0.4204808244276293), (17501, -0.4455469058889194), (17695, 0.2973056261082591), (18372, 0.5988781542071417), (32298, 0.3222129681246334), (38154, 0.0863222805347722), (38957, 0.0), (39142, -0.5287121980644168), (40145, -0.1427043271854564), (50313, 0.3162742705888221), (51033, -0.1771265363222767), (57117, 0.26042103761109453), (59122, -0.09935326726564145), (63839, 0.1890785550049905), (73669, 0.26040011362917104), (73925, 0.0), (74551, -0.029009890432391605), (77738, -0.1312164661257203), (84039, -0.09579347900399962), (88296, 0.0

In [32]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,-0.047395,2569
1,-0.165918,36946
2,0.431033,96884
3,0.280275,112293
4,-0.261032,138980


#### The top x similar users to input user
Now let's get the top 50 users that are most similar to the input.

In [33]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
20318,1.0,73116
42058,1.0,73822
38017,1.0,41785
59337,1.0,210331
43605,1.0,86226


Now, let's start recommending movies to the input user.

#### Rating of selected users to all movies
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our __pearsonDF__ from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [34]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,73116,110,3.5
1,1.0,73116,293,4.5
2,1.0,73116,527,5.0
3,1.0,73116,597,3.0
4,1.0,73116,1721,3.5


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [35]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,73116,110,3.5,3.5
1,1.0,73116,293,4.5,4.5
2,1.0,73116,527,5.0,5.0
3,1.0,73116,597,3.0,3.0
4,1.0,73116,1721,3.5,3.5


In [36]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,21.0,80.5
2,1.0,4.0
3,1.0,4.0
5,1.0,1.5
6,2.0,7.5


In [37]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
1,3.833333,1
2,4.000000,2
3,4.000000,3
5,1.500000,5
6,3.750000,6


Now let's sort it and see the top 20 movies that the algorithm recommended!

In [38]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
85261,5.0,85261
96610,5.0,96610
4856,5.0,4856
94752,5.0,94752
4499,5.0,4499
6650,5.0,6650
95105,5.0,95105
95170,5.0,95170
3868,5.0,3868
96069,5.0,96069


In [39]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
3776,3868,"Naked Gun: From the Files of Police Squad!, The",1988
4405,4499,Dirty Rotten Scoundrels,1988
4761,4856,Destiny in Space,1994
6541,6650,Kind Hearts and Coronets,1949
16884,85261,Mars Needs Moms,2011
19085,94752,Gifted Hands: The Ben Carson Story,2009
19174,95105,Madagascar 3: Europe's Most Wanted,2012
19194,95170,Beauty and the Beast: The Enchanted Christmas,1997
19384,96069,"Little Mermaid: Ariel's Beginning, The",2008
19523,96610,Looper,2012


### Advantages and Disadvantages of Collaborative Filtering

##### Advantages
* Takes other user's ratings into consideration
* Doesn't need to study or extract information from the recommended item
* Adapts to the user's interests which might change over time

##### Disadvantages
* Approximation function can be slow
* There might be a low of amount of users to approximate
* Privacy issues when trying to learn the user's preferences